<a href="https://colab.research.google.com/github/dasmiq/CS6120-HW5/blob/main/BeamSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation with pre-trained Transformers
In this assignment we will work with Pre-trained Transformers such as GPT2 for generating text from a given sequence. Transformers aim to address the long term dependency issue in sequence-to-seuqence prediction by using concepts such as self-attention and positional encoding. GPT2 is a langauge model, pretrained on text generation, that can be used as a multi-task learner for tasks such as summarization, question-answering, and other generation tasks. This assignment's focus is on using GPT2 to generate text via greedy decoding and beam search. For more background on beam search, see [Jurafsky and Martin, chapter 11](https://web.stanford.edu/~jurafsky/slp3/11.pdf).

In [5]:
!pip install transformers

In [6]:
import copy
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [8]:
sentences = ['I like walking and', 
             'Martha wanted to read a book that',
             'Thomas is studying computer science to',
             'Their friendship inspired',
             'We should take the trash out since',
             'I am not a fan of coffee because',
             'I could not complete my homework by the deadline because',
             'The last semester was much easier due to',
             'I will be painting the walls white so that'
            ]

We apply greedy decoding to get predictions for each sentence here. This function returns the text output of greedy decoding. Modify it to return a tuple (ordered pair) of text and average log-likelihood per word for each sentence.

In [9]:
## TODO: Modify this function to return pairs of text and average log-likelihood
## per word for each sentence.
def greedy_decode(sentences, max_length, tokenizer):
  # Obtain loss from output and calculate
  # log likelihood for each sentence
  final_dict = {}
  texts = []
  for sentence in sentences:
    predicted_sentence = copy.copy(sentence)
    new_dict = {}
    # Predict a word each itertaion until the max length
    for i in range(max_length):
      indexed_tokens = tokenizer.encode(predicted_sentence)
      token_tensors = torch.tensor([indexed_tokens])

      with torch.no_grad():
        output = model(token_tensors, labels=token_tensors)
        predictions = output[1]

      predicted_index = torch.argmax(predictions[0, -1, :]).item()
      ## Average log likelihood for each sentence
      likelihood_avg = predictions[0, -1, :][predicted_index]/len(indexed_tokens)
      predicted_sentence = tokenizer.decode(indexed_tokens + [predicted_index])
      new_dict[predicted_sentence] = likelihood_avg.numpy()
    texts.append(predicted_sentence)
    final_dict[predicted_sentence] = new_dict
  return texts, final_dict

In [10]:
texts,sentence_with_likelihood = greedy_decode(sentences, max_length=25, tokenizer=tokenizer)
sentence_with_likelihood

{'I am not a fan of coffee because it is not good for you. I am a fan of coffee because it is not good for you.\n\n\nI': {'I am not a fan of coffee because it': array(-11.167954, dtype=float32),
  'I am not a fan of coffee because it is': array(-11.261746, dtype=float32),
  'I am not a fan of coffee because it is not': array(-11.447308, dtype=float32),
  'I am not a fan of coffee because it is not good': array(-11.8024, dtype=float32),
  'I am not a fan of coffee because it is not good for': array(-7.6146045, dtype=float32),
  'I am not a fan of coffee because it is not good for you': array(-7.437751, dtype=float32),
  'I am not a fan of coffee because it is not good for you.': array(-5.6716495, dtype=float32),
  'I am not a fan of coffee because it is not good for you. I': array(-10.084319, dtype=float32),
  'I am not a fan of coffee because it is not good for you. I am': array(-9.743275, dtype=float32),
  'I am not a fan of coffee because it is not good for you. I am a': array(-6.109

You'll be implementing **beam search**, which returns a list of the $k$ most likely output sequences for each sentence. For this assignment, let $k = 8$. For the first token in the generated text, you will select the top $k$ output tokens. Then, for the next token, find the $k$-best continuations for each of those $k$ hypotheses and select the $k$-best overall. Return the $k$-best overall hypotheses according to average log likelihood per word. Note that if we don't average per word, the decoder will simply prefer shorter outputs. As above, return tuples of text output and avergae log likelihood.

In [15]:
## TODO: Implement beam search.
import queue as q

def beam_search(sentences, max_length, tokenizer, k=8):
  # Your code here
  # Beam search should be implemented 
  # without the use of model.generate().
  # Calculate average per-word log likelihood for each
  # output sequence.
  texts= []
  scores = []
  for sentence in sentences:
    predicted_texts = []
    new_li= []
    predicted_sentence = copy.copy(sentence)
    temp_var = k
    top_k = q.PriorityQueue()
    top_k.put((0, predicted_sentence))
    for i in range(max_length):
      qu = q.PriorityQueue()
      while not top_k.empty():
        (val, predicted_sentence) = top_k.get()
        indexed_tokens = tokenizer.encode(predicted_sentence)
        token_tensors = torch.tensor([indexed_tokens])
        with torch.no_grad():
          output = model(token_tensors, labels=token_tensors)
          predictions = output[1]
        sort_predict = predictions[0, -1, :].sort()

        for j in range(1, temp_var + 1):
          sorted_prediction_value = sort_predict.values[-j].item()
          sorted_prediction_index = sort_predict.indices[-j].item()
          predicted_word = tokenizer.decode([sorted_prediction_index])
          if predicted_word == '<|endoftext|>':
            temp_var = temp_var - 1
            predicted_sentence = tokenizer.decode(indexed_tokens)
            predicted_texts.append(predicted_sentence)
            new_li.append((predicted_sentence, val))
            continue
          predicted_sentence = tokenizer.decode(indexed_tokens + [sorted_prediction_index])
          likelihood_avg = predictions[0, -1, :][sorted_prediction_index]/len(indexed_tokens)
          new_li.append((predicted_sentence, likelihood_avg.item()))
          qu.put((-likelihood_avg.item(), predicted_sentence))
      for j in range(0, temp_var):
        top_k.put(qu.get())
    while not top_k.empty():
      (_, sentence) = top_k.get()
      predicted_texts.append(sentence)
    scores.append(new_li)
    texts.append(predicted_texts)
  return texts, scores


For further exploration, you can experiment with $k$ to see how the fluency of text changes.

In [16]:
texts, beam_scores = beam_search(sentences, max_length=25, tokenizer=tokenizer)

In [17]:
beam_scores

[[('I like walking and biking', -25.579164505004883),
  ('I like walking and I', -25.630165100097656),
  ('I like walking and running', -25.63140296936035),
  ('I like walking and talking', -25.665483474731445),
  ('I like walking and playing', -25.824312210083008),
  ('I like walking and doing', -25.869110107421875),
  ('I like walking and going', -25.8880558013916),
  ('I like walking and walking', -25.894426345825195),
  ('I like walking and biking,', -20.663795471191406),
  ('I like walking and biking.', -20.703022003173828),
  ('I like walking and biking and', -20.713153839111328),
  ('I like walking and biking,"', -20.828983306884766),
  ('I like walking and biking to', -21.009517669677734),
  ('I like walking and biking in', -21.011016845703125),
  ('I like walking and biking."', -21.020587921142578),
  ('I like walking and biking but', -21.031265258789062),
  ('I like walking and I like', -26.970882415771484),
  ("I like walking and I'm", -27.40763282775879),
  ('I like walking